## Simple Example with Steam and Pressure Changer

This notebook demonstrates how to assemble, initialize, and run a very simple flowsheet using steam and a pressure changer. This breifly demonstrates the basics of setting up a unit model.  See the documentation for a more complete description of the framework.

## Imports
Standard Pyomo and IDAES core components are imported below along with Panda which will be used in this example to print a table of steam states in the unit model.

In [1]:
import pyomo.environ as pe
from idaes.core import FlowsheetBlock, StateBlockBase
from idaes.unit_models.pressure_changer import PressureChanger
from idaes.property_models import iapws95_ph
import pandas as pd

## Create a Unit model

The lines below create a Pyomo concrete model, pressure changer unit model and steam property parameter block instance. The steam property parameter block contains constants that can be shared among all similar state blocks in a unit model. The iapws (international association of properties of water and steam) property model is used.

In [2]:
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(default={"dynamic": False})
model.fs.properties = iapws95_ph.Iapws95ParameterBlock()

## Create a Pressure Changer Model

Here the pressure changer model is created. Four different types of pressure changer operations are considered;  
1. isothermal; constant temperature,
2. Adiabatic; net heat transfer of zero,
3. pump; assuming incompressible fluid,
4. isentropic; constant entropy.

We will test the isentropic assumption for this example.

In [3]:
model.fs.pressure_changer = PressureChanger(default={"property_package": model.fs.properties,
                            "thermodynamic_assumption":'isentropic'})

## Fix Inlets and Initialize

In [4]:
init_state = {
        "flow_mol":1e3,
        "pressure":1e6,
        "enth_mol":10000
    }
model.fs.pressure_changer.inlet[0].flow_mol.fix(1e3)
model.fs.pressure_changer.inlet[0].pressure.fix(1e6)
model.fs.pressure_changer.inlet[0].enth_mol.fix(10000)
model.fs.pressure_changer.deltaP.fix(500)
model.fs.pressure_changer.efficiency_isentropic.fix(0.9)

model.fs.pressure_changer.initialize(state_args=init_state)

## Solve the Disconnected Unit model

Solve the model after initialization

In [5]:
solver = pe.SolverFactory('ipopt')

In [6]:
solver.solve(model, tee=True)

Ipopt 3.12.9: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.9, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       23
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:        9
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        9
Total number of ineq

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 9, 'Number of variables': 9, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.9\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.0731346607208252}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## Write a Function to Show States at Ports

Write a function that will give a nice tabulated overview of the conditions in the process.

In [7]:
def state_table(m):
    head = ["State Block", 
        "Flow (mol/s)",
        "Temperature (K)",
        "Pressure (Pa)",
        "Enthalpy (J/mol)",
        "Vapor Fraction"]
    st = pd.DataFrame(columns=head)
    j = 0
    for c in model.component_objects():
        if isinstance(c, StateBlockBase):
            for i in c:
                row = [c[i].name, 
                       pe.value(c[i].flow_mol), 
                       pe.value(c[i].temperature), 
                       pe.value(c[i].pressure), 
                       pe.value(c[i].enth_mol),
                       pe.value(c[i].vapor_frac)]
                st.loc[j] = row
                j += 1
    return st

## Show States

The table below shows the properties of the state blocks created by the pressure changer unit model.

In [8]:
state_table(model)

,State Block,Flow (mol/s),Temperature (K),Pressure (Pa),Enthalpy (J/mol),Vapor Fraction
0,fs.pressure_changer.properties_isentropic[0.0],1000.0,405.076005,1001000.25,10000.020380,0.0
1,fs.pressure_changer.control_volume.properties_...,1000.0,405.075897,1000000.00,10000.000000,0.0
2,fs.pressure_changer.control_volume.properties_...,1000.0,405.075958,1000500.00,10000.010726,0.0
